In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
DEFAULT_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


conducting subsequent work with the following platform: CUDA


In [2]:
def test_nb_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy_1 = copy.deepcopy(htf)
    htf_copy_2 = copy.deepcopy(htf)
   
    # Get system and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    system_copy_1 = copy.deepcopy(system)
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)

    # Get nb force
    nb_force_index = 3 
    nb_force = system.getForce(nb_force_index) 

    # Get system and positions
    hybrid_system = htf.hybrid_system
    hybrid_positions = htf.hybrid_positions

    # Get custom nb force 
    custom_electrostatics_nb_force_index = 3 if not is_solvated else 4
    custom_sterics_nb_force_index = 4 if not is_solvated else 5
    custom_nb_exceptions_force_index = 5 if not is_solvated else 6
    standard_nb_force_index = 6 if not is_solvated else 7
       
    custom_electrostatics_nb_force = hybrid_system.getForce(custom_electrostatics_nb_force_index)
    custom_sterics_nb_force = hybrid_system.getForce(custom_sterics_nb_force_index)
    custom_nb_exceptions_force = hybrid_system.getForce(custom_nb_exceptions_force_index)
#     standard_nb_force = hybrid_system.getForce(standard_nb_force_index)
    standard_nb_force = htf_copy_1.hybrid_system.getForce(standard_nb_force_index)

    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    lambda_old_str = ['lambda_alchemical_electrostatics_old', 'lambda_alchemical_sterics_old']
    lambda_new_str = ['lambda_alchemical_electrostatics_new', 'lambda_alchemical_sterics_new']
    lambda_old_exceptions_str = ['lambda_alchemical_electrostatics_exceptions_old', 'lambda_alchemical_sterics_exceptions_old']
    lambda_new_exceptions_str = ['lambda_alchemical_electrostatics_exceptions_new', 'lambda_alchemical_sterics_exceptions_new']
    
    for custom_nb_force in [custom_electrostatics_nb_force, custom_sterics_nb_force]:
        for i in range(custom_nb_force.getNumGlobalParameters()):
            if custom_nb_force.getGlobalParameterName(i) in lambda_old_str:
                custom_nb_force.setGlobalParameterDefaultValue(i, lambda_old)
            elif custom_nb_force.getGlobalParameterName(i) in lambda_new_str:
                custom_nb_force.setGlobalParameterDefaultValue(i, lambda_new)

    for i in range(custom_nb_exceptions_force.getNumGlobalParameters()):
        if custom_nb_exceptions_force.getGlobalParameterName(i) in lambda_old_exceptions_str:
            custom_nb_exceptions_force.setGlobalParameterDefaultValue(i, lambda_old)
        elif custom_nb_exceptions_force.getGlobalParameterName(i) in lambda_new_exceptions_str:
            custom_nb_exceptions_force.setGlobalParameterDefaultValue(i, lambda_new)
                                 
    for i in range(standard_nb_force.getNumGlobalParameters()):
        if standard_nb_force.getGlobalParameterName(i) == 'lambda_alchemical_electrostatics_reciprocal':
            standard_nb_force.setGlobalParameterDefaultValue(i, lambda_new)
       
    # Get energy components of standard nb force (direct space only)
    nb_force.setReciprocalSpaceForceGroup(31)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta, platform=DEFAULT_PLATFORM)
    print(components_other)
    
    # Get energy components of custom nb force + custom nb force + custom bond force
    hybrid_system.removeForce(standard_nb_force_index)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta, platform=DEFAULT_PLATFORM)
    print(components_hybrid)
    
    for k, v in context_hybrid.getParameters().items():
        print(k, v)
    
    print([components_other[nb_force_index][1]], [np.sum([energy[1] for i, energy in enumerate(components_hybrid) if i in [3, 4, 5] ])])
    assert np.isclose([components_other[nb_force_index][1]], np.sum([energy[1] for i, energy in enumerate(components_hybrid) if i in [3, 4, 5] ])), f"standard nonbonded: {components_other[nb_force_index][1]}, custom nonbonded/bond: {np.sum([energy[1] for i, energy in enumerate(components_hybrid) if i in [3, 4, 5] ])}"
    
     # Get energy components of standard nb force (reciprocal space only) (og system)
    system_copy_1.getForce(nb_force_index).setIncludeDirectSpace(False)
    thermostate_other = ThermodynamicState(system=system_copy_1, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other_reciprocal = compute_potential_components(context_other, beta=beta, platform=DEFAULT_PLATFORM)
    print(components_other_reciprocal)
    
    thermostate_hybrid = ThermodynamicState(system=htf_copy_1.hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid_reciprocal = compute_potential_components(context_hybrid, beta=beta, platform=DEFAULT_PLATFORM)
    print(components_hybrid_reciprocal)

    print([components_other_reciprocal[nb_force_index][1]], [components_hybrid_reciprocal[6][1]])
    assert np.isclose([components_other_reciprocal[nb_force_index][1]], [components_hybrid_reciprocal[6][1]]), f"standard nonbonded: {components_other_reciprocal[nb_force_index][1]}, custom nonbonded/bond: {components_hybrid_reciprocal[0][1]}"

    
    print(f"Success! Custom nb force and standard nb force energies are equal!")

    if check_scale:
        
        # Get custom nb and custom bond forces and hybrid positions
        hybrid_system = htf_copy_2.hybrid_system
        hybrid_positions = htf_copy_2.hybrid_positions
        
        # Get custom nb force 
        custom_electrostatics_nb_force_index = 3 if not is_solvated else 4
        custom_sterics_nb_force_index = 4 if not is_solvated else 5
        custom_nb_exceptions_force_index = 5 if not is_solvated else 6
        standard_nb_force_index = 6 if not is_solvated else 7

        custom_electrostatics_nb_force = hybrid_system.getForce(custom_electrostatics_nb_force_index)
        custom_sterics_nb_force = hybrid_system.getForce(custom_sterics_nb_force_index)
        custom_nb_exceptions_force = hybrid_system.getForce(custom_nb_exceptions_force_index)

        # Get energy components of custom nb force + custom nb force + custom bond force
        hybrid_system.removeForce(standard_nb_force_index)
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta, platform=DEFAULT_PLATFORM)
        print(components_hybrid)

        # Set rest lambda to 0.5
        for custom_nb_force in [custom_electrostatics_nb_force, custom_sterics_nb_force]:
            for i in range(custom_nb_force.getNumGlobalParameters()):
                if custom_nb_force.getGlobalParameterName(i) == 'lambda_rest_electrostatics':
                    custom_nb_force.setGlobalParameterDefaultValue(i, 0.5)
                elif custom_nb_force.getGlobalParameterName(i) == 'lambda_rest_sterics':
                    custom_nb_force.setGlobalParameterDefaultValue(i, 0.5)
                
        for i in range(custom_nb_exceptions_force.getNumGlobalParameters()):
            if custom_nb_exceptions_force.getGlobalParameterName(i) == 'lambda_rest_electrostatics_exceptions':
                custom_nb_exceptions_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_nb_exceptions_force.getGlobalParameterName(i) == 'lambda_rest_sterics_exceptions':
                custom_nb_exceptions_force.setGlobalParameterDefaultValue(i, 0.5)
            
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta, platform=DEFAULT_PLATFORM)
        print(components_hybrid_scaled)

        print([np.sum([energy[1] for i, energy in enumerate(components_hybrid) if i in [3, 4, 5] ])], [np.sum([energy[1] for i, energy in enumerate(components_hybrid_scaled) if i in [3, 4, 5] ])])
        assert not np.isclose([np.sum([energy[1] for i, energy in enumerate(components_hybrid) if i in [3, 4, 5] ])], [np.sum([energy[1] for i, energy in enumerate(components_hybrid_scaled) if i in [3, 4, 5] ])]), f"standard nonbonded: {components_other[nb_force_index][1]}, custom nonbonded/bond: {np.sum([energy[1] for i, energy in enumerate(components_hybrid) if i in [3, 4, 5] ])}"

        print(f"Success! Scaling the custom nb and bond forces changes the energy")
        

# Alanine dipeptide in vacuum

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [27]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [28]:
test_nb_energies(htf)


conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414457), ('HarmonicAngleForce', 0.6072023625714801), ('PeriodicTorsionForce', 16.176828968049836), ('NonbondedForce', -39.186928227296285), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.19661163482151217), ('CustomAngleForce', 260.92795488030396), ('CustomTorsionForce', 27.47798844472086), ('CustomNonbondedForce', -134.4041837676869), ('CustomNonbondedForce', 4.716822222623423), ('CustomBondForce', 90.50042955016903), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414457), ('HarmonicAngleForce', 0.6072023625714801), ('PeriodicTorsionForce', 16.176828968049836), ('NonbondedForce', 0.0), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.19661163482151217), ('CustomAngleForce', 260.92795488

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [25]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [26]:
test_nb_energies(htf, is_old=False)

conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.19661163482151217), ('HarmonicAngleForce', 260.92777677949186), ('PeriodicTorsionForce', 27.47509191436743), ('NonbondedForce', -88.2896603063111), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.19661163482151217), ('CustomAngleForce', 260.92795488030396), ('CustomTorsionForce', 27.47798844472086), ('CustomNonbondedForce', -89.72759295871141), ('CustomNonbondedForce', 9.277896982930873), ('CustomBondForce', -7.83996894169944), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.19661163482151217), ('HarmonicAngleForce', 260.92777677949186), ('PeriodicTorsionForce', 27.47509191436743), ('NonbondedForce', 0.0), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.19661163482151217), ('CustomAngleForce', 260.9279548803

### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [33]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [34]:
test_nb_energies(htf, check_scale=True)


conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414457), ('HarmonicAngleForce', 0.6072023625714801), ('PeriodicTorsionForce', 16.176828968049836), ('NonbondedForce', -39.186928227296285), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.11204966907978851), ('CustomAngleForce', 7.754114928422474), ('CustomTorsionForce', 26.2414559977149), ('CustomNonbondedForce', -134.4041837676869), ('CustomNonbondedForce', 4.716822222623423), ('CustomBondForce', 90.50042955016903), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414457), ('HarmonicAngleForce', 0.6072023625714801), ('PeriodicTorsionForce', 16.176828968049836), ('NonbondedForce', 0.0), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.11204966907978851), ('CustomAngleForce', 7.754114928422

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [37]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [38]:
test_nb_energies(htf, is_old=False, check_scale=True)


conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.11204966907978851), ('HarmonicAngleForce', 7.753936827610332), ('PeriodicTorsionForce', 26.23856046443492), ('NonbondedForce', 551.2600335631639), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.11204966907978851), ('CustomAngleForce', 7.754114928422474), ('CustomTorsionForce', 26.2414559977149), ('CustomNonbondedForce', -97.94581621939766), ('CustomNonbondedForce', 641.5552191005575), ('CustomBondForce', 7.650297172428192), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.11204966907978851), ('HarmonicAngleForce', 7.753936827610332), ('PeriodicTorsionForce', 26.23856046443492), ('NonbondedForce', 0.0), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.11204966907978851), ('CustomAngleForce', 7.754114928422474)

# Alanine dipeptide in solvent

### Test one alchemical region, no scale regions

In [4]:
# from perses.annihilation.relative import RestCapablePMEHybridTopologyFactory

In [5]:
# # Create a htf
# atp, system_generator = generate_atp(phase = 'solvent')
# topology_proposal, new_positions, _, _ = generate_dipeptide_top_pos_sys(atp.topology, 
#                                'THR', 
#                                atp.system, 
#                                atp.positions, 
#                                system_generator,
#                                flatten_torsions=True,
#                                flatten_exceptions=True,
#                                validate_endstate_energy=False)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]


making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 6
INFO:geometry:Atom index proposal order is [14, 18, 15, 16, 19, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, s

generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (19, 18, 10, 8, [1, Quantity(value=-0.07858019216845058, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 5]), 1: (19, 18, 10, 14, [1, Quantity(value=2.118586571250296, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 5]), 2: (19, 18, 10, 13, [1, Quantity(value=-2.149781203448759, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 5]), 3: (8, 10, 14, 17, [1, Quantity(value=-2.1249532223845904, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 6]), 4: (8, 10, 14, 16, [1, Quantity(value=2.0543043017157916, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 4]), 5: (8, 10, 14, 15, [1, Quantity(value=-0.02223392579985184, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 3]), 6: (18, 10, 14, 17, [1, Quantity(value=2.0255587336734213, unit=radian), Quantity(valu

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.815513850992346


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 4.780760226787146
INFO:geometry:final reduced energy 21.596273881350857
INFO:geometry:sum of energies: 21.59627407777949
INFO:geometry:magnitude of difference in the energies: 1.9642863513524844e-07
INFO:geometry:Final logp_proposal: -17931.575647370104


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.00017810081214392654), ('CustomTorsionForce', 0.002891174165036068), ('CustomBondForce', 4.777690951809965)]


In [6]:
# flatten_exceptions = True
# flatten_torsions = True
# endstate = None
# rest_region = None
# htf = RestCapablePMEHybridTopologyFactory(topology_proposal=topology_proposal,
#                      current_positions=atp.positions,
#                      new_positions=new_positions,
#                      rest_region=rest_region,
#                      generate_htf_for_testing=True,
#                      interpolate_old_and_new_14s=flatten_exceptions
#                     )

INFO:relative:*** Generating RestCapablePMEHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.918423065872431
INFO:relative:w_scale is 0.1
INFO:relative:Set w_scale to 0 for testing
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354

In [7]:
# with open("atp_solvent_nb.pickle", "wb") as f:
#     pickle.dump(htf, f)

#### Old system nb force vs hybrid system custom nb force and custom bond force (electrostatics)

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [50]:
# exceptions_force = htf.hybrid_system.getForce(6)
# for i in range(exceptions_force.getNumBonds()):
#     print(exceptions_force.getBondParameters(i))

In [9]:
# from simtk.openmm import XmlSerializer
# with open('hybrid_system.xml', 'w') as f:
#     f.write(XmlSerializer.serialize(htf.hybrid_system))

In [8]:
# with open("hybrid_positions.pickle", "wb") as f:
#     pickle.dump(htf.hybrid_positions, f)

In [10]:
# with open('old_system.xml', 'w') as f:
#     f.write(XmlSerializer.serialize(htf._topology_proposal.old_system))

In [12]:
# with open("old_positions.pickle", "wb") as f:
#     pickle.dump(htf.old_positions(htf.hybrid_positions), f)

In [6]:
test_nb_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03455284552741751), ('HarmonicAngleForce', 0.6071989411338462), ('PeriodicTorsionForce', 16.176841775340726), ('NonbondedForce', 48048.15874389563), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.28581688607525807), ('CustomAngleForce', 7.781404329601241), ('CustomTorsionForce', 26.48030777263283), ('CustomNonbondedForce', -7398.382023995852), ('CustomNonbondedForce', 7032.129149022988), ('CustomBondForce', 48414.4178835812), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
AndersenCollisionFrequency 1.0
AndersenTemperature 300.0
MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_alchemical_angles_new 0.0
lambda_alchemical_angles_old 1.0
lambda_alchemical_bonds_new 0.0
lambda_alchemical_bonds_old 1.0
lambda_alchemical_electrostatics_exceptions_new 0.0
lambda_alchemical_electrostatics_exce

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [6]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [7]:
test_nb_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.285816886075258), ('HarmonicAngleForce', 7.781226228912823), ('PeriodicTorsionForce', 26.47741160999302), ('NonbondedForce', 48377.94175328148), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.28581688607525807), ('CustomAngleForce', 7.781404329601241), ('CustomTorsionForce', 26.48030777263283), ('CustomNonbondedForce', -7383.797824099858), ('CustomNonbondedForce', 7321.555217409464), ('CustomBondForce', 48440.18369417566), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.2858168860752572), ('HarmonicAngleForce', 7.7812262289128125), ('PeriodicTorsionForce', 26.477411609993002), ('NonbondedForce', -49058.8156811369), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the follow

### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [18]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [19]:
test_nb_energies(htf, is_solvated=True, check_scale=True)


conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03455176254144657), ('HarmonicAngleForce', 0.6072023625714819), ('PeriodicTorsionForce', 16.176828968049854), ('NonbondedForce', 48577.510535772766), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.436358777060491), ('CustomAngleForce', 8.349984184707225), ('CustomTorsionForce', 24.706018116420218), ('CustomNonbondedForce', -7334.199200237913), ('CustomNonbondedForce', 7780.49610808769), ('CustomBondForce', 48131.21520351297), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
AndersenCollisionFrequency 1.0
AndersenTemperature 300.0
MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_alchemical_angles_new 0.0
lambda_alchemical_angles_old 1.0
lambda_alchemical_bonds_new 0.0
lambda_alchemical_bonds_old 1.0
lambda_alchemical_electrostatics_exceptions_new 0.0
lambda_alchemical_electrostatics_exce

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [21]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [22]:
test_nb_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.436358777060491), ('HarmonicAngleForce', 8.349806083894475), ('PeriodicTorsionForce', 24.701676520795253), ('NonbondedForce', 48616.250012431265), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.436358777060491), ('CustomAngleForce', 8.349984184707225), ('CustomTorsionForce', 24.706018116420218), ('CustomNonbondedForce', -7325.6994711743655), ('CustomNonbondedForce', 7799.613938465385), ('CustomBondForce', 48142.33716493503), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
AndersenCollisionFrequency 1.0
AndersenTemperature 300.0
MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_alchemical_angles_new 0.0
lambda_alchemical_angles_old 1.0
lambda_alchemical_bonds_new 0.0
lambda_alchemical_bonds_old 1.0
lambda_alchemical_electrostatics_exceptions_new 1.0
lambda_alchemical_electrostatics_excep

# CDK2 transformation in solvent

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [54]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [55]:
test_nb_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.5773624247683875), ('HarmonicAngleForce', 29.004860118481027), ('PeriodicTorsionForce', 14.976236339333706), ('NonbondedForce', 66770.07027409678), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 30.676180764366656), ('CustomTorsionForce', 15.015954434573812), ('CustomNonbondedForce', -11618.580433337456), ('CustomNonbondedForce', 14488.970730343253), ('CustomBondForce', 63899.68070530242), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.5773624247683875), ('HarmonicAngleForce', 29.004860118481027), ('PeriodicTorsionForce', 14.976236339333706), ('NonbondedForce', -65203.97895621224), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [56]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [57]:
test_nb_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.489378129340331), ('HarmonicAngleForce', 30.630126666874034), ('PeriodicTorsionForce', 15.011200335725704), ('NonbondedForce', 66782.47266746471), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 30.676180764366656), ('CustomTorsionForce', 15.015954434573812), ('CustomNonbondedForce', -11617.495598709871), ('CustomNonbondedForce', 14493.240231208783), ('CustomBondForce', 63906.728789225155), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.489378129340331), ('HarmonicAngleForce', 30.630126666874034), ('PeriodicTorsionForce', 15.011200335725704), ('NonbondedForce', -65204.787336655856), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the

### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [59]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [60]:
test_nb_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.5773624247683875), ('HarmonicAngleForce', 29.004860118481027), ('PeriodicTorsionForce', 14.976236339333706), ('NonbondedForce', 66433.40086528676), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.800334653456577), ('CustomTorsionForce', 14.98233372968621), ('CustomNonbondedForce', -11682.245408114439), ('CustomNonbondedForce', 14215.963208043462), ('CustomBondForce', 63899.68358862266), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.5773624247683875), ('HarmonicAngleForce', 29.004860118481027), ('PeriodicTorsionForce', 14.976236339333706), ('NonbondedForce', -65124.03514213234), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the 

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [61]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/31_rest_over_protocol/cdk2_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [62]:
test_nb_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.489378129340331), ('HarmonicAngleForce', 29.754280555963955), ('PeriodicTorsionForce', 14.977580108775431), ('NonbondedForce', 66445.4899755582), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.5773624247683875), ('CustomAngleForce', 29.800334653456577), ('CustomTorsionForce', 14.98233372968621), ('CustomNonbondedForce', -11681.527447749873), ('CustomNonbondedForce', 14220.23056906056), ('CustomBondForce', 63906.78740313257), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.489378129340331), ('HarmonicAngleForce', 29.754280555963955), ('PeriodicTorsionForce', 14.977580108775431), ('NonbondedForce', -65131.244084710044), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the fol

## barnase:barstar (no naked charge fix, no counterion fix)

### Test one alchemical region, no scale regions

#### Old system nonbondeds vs hybrid system nonbondeds

In [26]:
# Load htf
with open("bnbs_apo.pickle", "rb") as f:
    htf = pickle.load(f)
    

In [27]:
test_nb_energies(htf, is_solvated=True)


conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 235.19170527285917), ('HarmonicAngleForce', 617.0801971492558), ('PeriodicTorsionForce', 1827.6228592530676), ('NonbondedForce', 57571456.6629012), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 235.19170527285917), ('CustomAngleForce', 618.469614075649), ('CustomTorsionForce', 1827.650565186107), ('CustomNonbondedForce', -79221.28149200724), ('CustomNonbondedForce', 57203102.40806319), ('CustomBondForce', 447603.8036728785), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
AndersenCollisionFrequency 1.0
AndersenTemperature 300.0
MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_alchemical_angles_new 0.0
lambda_alchemical_angles_old 1.0
lambda_alchemical_bonds_new 0.0
lambda_alchemical_bonds_old 1.0
lambda_alchemical_electrostatics_exceptions_new 0.0
lambda_alchemical_electrostatics_exceptio

#### New system nonbondeds vs hybrid system nonbondeds

In [10]:
# Load htf
with open("bnbs_apo.pickle", "rb") as f:
    htf = pickle.load(f)

In [11]:
test_nb_energies(htf, is_old=False, is_solvated=True)


conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 234.85763005447197), ('HarmonicAngleForce', 616.116462838331), ('PeriodicTorsionForce', 1817.399334662456), ('NonbondedForce', 57572024.87828817), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 235.19170527285917), ('CustomAngleForce', 617.8183673912378), ('CustomTorsionForce', 1827.6515646557948), ('CustomNonbondedForce', -79013.15928562051), ('CustomNonbondedForce', 57203475.60907662), ('CustomBondForce', 447590.696971324), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
AndersenCollisionFrequency 1.0
AndersenTemperature 300.0
MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_alchemical_angles_new 0.0
lambda_alchemical_angles_old 1.0
lambda_alchemical_bonds_new 0.0
lambda_alchemical_bonds_old 1.0
lambda_alchemical_electrostatics_exceptions_new 1.0
lambda_alchemical_electrostatics_exceptio

In [ ]:
# TO DO: actually test the scaling of the energies matches